In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
%matplotlib inline
color = sns.color_palette()
from pandas.plotting import table
import scipy
from scipy.stats import pearsonr, spearmanr
from wordcloud import WordCloud, STOPWORDS
import datetime as dt

In [ ]:
# Load data
don = pd.read_csv("data/donations.csv")
ess = pd.read_csv("data/essays.csv")
out = pd.read_csv("data/outcomes.csv")
res = pd.read_csv("data/resources.csv")
pro = pd.read_csv("data/projects.csv")

# Build design matrix

## Donation features

In [ ]:
# message word count
don['message_num_words'] = don['donation_message'].str.count(' ') + 1

In [ ]:
don = don[['donationid', 'projectid', 'donor_acctid', 
           'is_teacher_acct', 'donation_timestamp', 'donation_to_project',
           'message_num_words']]

In [ ]:
don['donation_timestamp'] = pd.to_datetime(don['donation_timestamp'])
pro['date_posted'] = pd.to_datetime(pro['date_posted'])

In [ ]:
don_date_posted = don.merge(pro[['projectid', 'date_posted']], on='projectid')

In [ ]:
from datetime import timedelta
don_date_posted['date_cutoff'] = don_date_posted['date_posted'] + timedelta(days=7)

In [ ]:
don_date_posted = don_date_posted.query('donation_timestamp < date_cutoff')

In [ ]:
don_cutoff_by_donor = (don_date_posted
                       .groupby(['projectid', 'donor_acctid'])
                       .agg({'is_teacher_acct': 'first',
                             'donation_to_project': 'sum',
                             'message_num_words': 'mean'})
                       .reset_index()
                       .fillna(0))

In [ ]:
don_cutoff_by_donor['is_teacher_acct'] = (don_cutoff_by_donor['is_teacher_acct'] == 't')

In [ ]:
don_features = (don_cutoff_by_donor
                .groupby('projectid')
                .agg({'donor_acctid': 'count',
                      'is_teacher_acct': 'sum',
                      'donation_to_project': 'sum',
                      'message_num_words': 'mean'})
                .rename(columns={'donor_acctid': 'num_donors',
                                 'is_teacher_acct': 'num_teachers', 
                                 'donation_to_project': 'total_donation_amount',
                                 'message_num_words': 'avg_msg_num_words'}))

## Writing features

In [ ]:
ess.head(1)

,projectid,teacher_acctid,title,short_description,need_statement,essay
0,ffffc4f85b60efc5b52347df489d0238,c24011b20fc161ed02248e85beb59a90,iMath,It is imperative that teachers bring technolog...,My students need four iPods.,I am a fourth year fifth grade math teacher. T...


In [ ]:
ess['title_num_words'] = ess['title'].str.count(' ') + 1
ess['description_num_words'] = ess['short_description'].str.count(' ') + 1
ess['statement_num_words'] = ess['need_statement'].str.count(' ') + 1
ess['essay_num_words'] = ess['essay'].str.count(' ') + 1

In [ ]:
ess = ess[['projectid', 'teacher_acctid', 
           'title_num_words', 'description_num_words', 
           'statement_num_words', 'essay_num_words']]

## Project features

In [ ]:
pro.head(1)

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,resource_type,poverty_level,grade_level,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted
0,316ed8fb3b81402ff6ac8f721bb31192,42d43fa6f37314365d08692e08680973,c0e6ce89b244764085691a1b8e28cb81,6.362701e+10,36.57634,-119.608713,Selma,CA,93662.0,NaN,...,Books,highest poverty,Grades 6-8,30.0,555.81,653.89,32.0,f,f,2014-05-12


In [ ]:
pro = pro[['projectid', 'teacher_acctid', 'schoolid', 
           'school_state', 'school_metro', 'school_charter',
           'teacher_prefix', 'teacher_teach_for_america', 'primary_focus_subject',
           'resource_type', 'poverty_level', 'grade_level', 
           'total_price_excluding_optional_support', 'students_reached',
           'date_posted']]

## Outcome info

In [ ]:
out.head(1)

,projectid,is_exciting,at_least_1_teacher_referred_donor,fully_funded,at_least_1_green_donation,great_chat,three_or_more_non_teacher_referred_donors,one_non_teacher_referred_donor_giving_100_plus,donation_from_thoughtful_donor,great_messages_proportion,teacher_referred_count,non_teacher_referred_count
0,ffffc4f85b60efc5b52347df489d0238,f,NaN,f,NaN,f,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
out = out[['projectid', 'fully_funded']]

In [ ]:
pro.shape, out.shape

((664098, 15), (619326, 2))

# Combine features

In [ ]:
# Merge project features and writing features
main = pd.merge(pro, ess, on=['projectid', 'teacher_acctid'])

In [ ]:
# Add donation features
main = pd.merge(main, don_features, on='projectid', how='left')

In [ ]:
main.shape

(664098, 23)

In [ ]:
main[['num_donors', 'num_teachers', 
      'total_donation_amount', 'avg_msg_num_words']] = \
    (main[['num_donors', 'num_teachers', 
           'total_donation_amount', 'avg_msg_num_words']]
     .fillna(0))

In [ ]:
# Add outcome
main = pd.merge(main, out, on='projectid')

In [ ]:
main.to_csv('./data/main_intermediate.csv', index=False)

In [ ]:
# Load data
main = pd.read_csv('./data/main_intermediate.csv')
pro = pd.read_csv("data/projects.csv")
out = pd.read_csv("data/outcomes.csv")[['projectid', 'fully_funded']]

## Add compound features
- day_of_week_posted
- school_success_rate
- teacher_success_rate
- percent_fulfilled
- amount_remaining

In [ ]:
# Add more features
main['date_posted'] = pd.to_datetime(main['date_posted'])
main['day_of_week_posted'] = main['date_posted'].dt.dayofweek

In [ ]:
main['percent_fulfilled'] = (main['total_donation_amount'] / main['total_price_excluding_optional_support'])
main['amount_remaining'] = main['total_price_excluding_optional_support'] - main['total_donation_amount']

In [ ]:
pro['date_posted'] = pd.to_datetime(pro['date_posted'])
out['fully_funded'] = (out['fully_funded'] == 't')
pro_out = pro.merge(out, on='projectid')

In [ ]:
# Query past projects for success rate
from datetime import timedelta
main['query_date'] = main['date_posted'] + timedelta(days=7) - timedelta(days=(4*30))
main['stop_date'] = main['date_posted'] + timedelta(days=7) - timedelta(days=365)
main_queries = (main[['projectid', 'teacher_acctid', 'query_date', 'stop_date']]
                .merge(pro_out[['projectid', 'teacher_acctid', 'date_posted', 'fully_funded']],
                       on='teacher_acctid',
                       suffixes=('', '_queried'))
                .query('stop_date <= date_posted and date_posted < query_date')
                .groupby(['projectid', 'teacher_acctid'])
                .agg({'fully_funded': 'mean'})
                .rename(columns={'fully_funded': 'teacher_pct_funded'})
                .reset_index())

In [ ]:
main_queries = (main[['projectid', 'schoolid', 'query_date', 'stop_date']]
                .merge(pro_out[['projectid', 'schoolid', 'date_posted', 'fully_funded']],
                       on='schoolid',
                       suffixes=('', '_queried'))
                .query('stop_date <= date_posted and date_posted < query_date')
                .groupby(['projectid', 'schoolid'])
                .agg({'fully_funded': 'mean'})
                .rename(columns={'fully_funded': 'school_pct_funded'})
                .reset_index())
main = (main
        .merge(main_queries, 
               on=['projectid', 'schoolid'], 
               how='left'))
main['school_pct_funded'] = main['school_pct_funded'].fillna(0)

In [ ]:
del main_queries

In [ ]:
main_queries = (main[['projectid', 'teacher_acctid', 'query_date', 'stop_date']]
                .merge(pro_out[['projectid', 'teacher_acctid', 'date_posted', 'fully_funded']],
                       on='teacher_acctid',
                       suffixes=('', '_queried'))
                .query('stop_date <= date_posted and date_posted < query_date')
                .groupby(['projectid', 'teacher_acctid'])
                .agg({'fully_funded': 'mean'})
                .rename(columns={'fully_funded': 'teacher_pct_funded'})
                .reset_index())
main = (main
        .merge(main_queries, 
               on=['projectid', 'teacher_acctid'], 
               how='left'))
main['teacher_pct_funded'] = main['teacher_pct_funded'].fillna(0)

In [ ]:
main.drop(columns=['query_date', 'stop_date'], inplace=True)

In [ ]:
main.to_csv('./processed_mastersheet1025.zip')

# Stop here

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
don_utility = don_pro.groupby('projectid').progress_apply(lambda x: x.loc[x.date <= x.x_days_after_date_posted]['donation_total'].sum())

100%|██████████| 525042/525042 [08:56<00:00, 978.88it/s]


In [ ]:
don_utility = don_utility.to_frame()
don_utility.columns = ['fund_before_thredshold']

In [ ]:
master = pd.merge(master,don_utility, left_on='projectid', right_on='projectid')

In [ ]:
master['percent_funding_fulfilled_before_threshold'] = master['fund_before_thredshold'] / master['total_price_excluding_optional_support']
master = master.drop('fund_before_thredshold',axis = 1)

In [ ]:
don_utility_1 = don_pro.groupby('projectid').progress_apply(lambda x: x.loc[x.date <= x.x_days_after_date_posted].sum())

100%|██████████| 525042/525042 [21:00<00:00, 479.69it/s]

In [ ]:
don_utility_1 = don_utility_1.to_frame()
don_utility_1.columns = ['num_don_before_threshold']

In [ ]:
master = pd.merge(master,don_utility_1, left_on='projectid', right_on='projectid')

In [ ]:
master.to_csv('processed_mastersheet0928.csv')